## Azure Machine Learning registries

__Requirements__ In order to benefit from this tutorial, you will need:

- An Azure account with an active subscription
- Installed Azure Machine Learning Python SDK v2 

__Motivations__ - This notebook explains how to create, get, and list a Registry using the Python SDK

## 1. Create a Registry

A registry is a container of shared assets that can be used across many workspaces.

In this section, you will create a yaml file with your registry parameters and an instance of `MLClient` to create a registry. 

### 1.1 Import the required libraries

In [ ]:
# import the required libraries
from azure.ai.ml import MLClient, load_registry
from azure.ai.ml.constants._common import LROConfigurations
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
import os

!pip show azure-ai-ml

### 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to registry. `DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios.

Reference for more available credentials if it does not work for you: [configure credential examples, azure-identity reference doc.](https://github.com/Azure/azureml-registries/blob/92aec1e16225ec37e43735ab560b599443ecb4be/registries/configuration.ipynb)

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

### 1.3 Create a MLclient 
You need a subscription id and resource group to in which the registry will be created. Update the following below and create the client. 
* `<subscription-id>`
* `<resource-group>`

In [ ]:
# Create a client without specifying an existing registry or workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="<SUBSCRIPTION_ID>",
    resource_group_name="<RESOURCE_GROUP>",
)

### 1.4 Review or create the registry definition in YAML
See https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-registries to understand the attributes for registry. You need to define:
* Name
* Optional description and tags
* Primary region 
* Optional additional regions for replication
* Optional storage account configuration for each region

Sample YAML files are available at [../../../../cli/resources/registry//](../../../../cli/resources/registry/)

In [ ]:
import sys

yaml_file = os.path.abspath(
    os.path.join(sys.path[0], "../../../../cli/resources/registry/registry.yml")
)

f = open(yaml_file, "r")
print(f.read())

### 1.5 Load yaml and create registry 

In [ ]:
registry = load_registry(source=yaml_file)
print(registry)

# You can override the registry name if needed
registry.name = "DemoRegistryPython"

registry = ml_client.registries.begin_create(registry=registry).result(
    timeout=LROConfigurations.POLLING_TIMEOUT
)

## 2. Get and List Registry



### 2.1 Get Registry



In [ ]:
reg_name = registry.name  # defined within yaml file
my_registry = ml_client.registries.get(name=reg_name)
print(my_registry)

### 2.1 List Registries

This will list all registries that exist under the resource group of your `MLClient`. 

In [ ]:
for reg in ml_client.registries.list():
    print(reg.name)